In [0]:
import pandas as pd
import plotly.express as px

# 1. Lê a tabela Gold como Pandas (rápido e perfeito para visualização)
df_raw = spark.sql("SELECT * FROM classes.gold.df_imunizacoes").toPandas()

# 2. Tratamento dos nomes
df_raw['federacao'] = (df_raw['federacao']
                       .str.title()
                       .str.replace('Rondonia', 'Rondônia')
                       .str.replace('Grande Do Norte', 'G. do Norte')
                       .str.replace('Grande Do Sul', 'G. do Sul')
                       .str.replace('Espirito Santo', 'Espírito Santo'))

# 3. Separa Brasil dos estados
df_brasil = df_raw[df_raw['numero_da_federacao'] == 'total'].copy()
df_estados = df_raw[df_raw['numero_da_federacao'] != 'total'].copy()

# 4. Colunas das vacinas (exclui identificação + colunas zeradas/problemáticas)
vacina_cols = [col for col in df_raw.columns 
               if col not in ['numero_da_federacao', 'federacao'] 
               and col not in ['sarampo', 'haemophilus_influenzae_b','dupla_adulto_dtpa_gestante',
                               'dtpa_gestante','tetravalente_dtp_hib_tetra','varicela']]  # essas estão zeradas

# 5. Calcula média geral e ordena estados do melhor ao pior
df_estados['media_geral'] = df_estados[vacina_cols].mean(axis=1)
df_estados = df_estados.sort_values('media_geral', ascending=False).round(1)

# 6. Prepara o dataframe do heatmap
df_heat = df_estados.set_index('federacao')[vacina_cols]

# 7. Nomes bonitos das vacinas (vai ficar lindo no gráfico)
nomes_vacinas = {
    'bcg': 'BCG',
    'hepatite_b_idade_menor_igual_30_dias': 'Hepatite B (<30d)',
    'rotavirus_humano': 'Rotavírus',
    'meningococo_c': 'Meningo C',
    'hepatite_b': 'Hepatite B',
    'penta': 'Penta',
    'pneumococica': 'Pneumocócica',
    'poliomielite': 'Poliomielite',
    'poliomielite_4_anos': 'Polio reforço 4 anos',
    'febre_amarela': 'Febre Amarela',
    'hepatite_a': 'Hepatite A',
    'pneumococica_1_ref': 'Pneumo reforço',
    'meningococo_c_1_ref': 'Meningo reforço',
    'poliomielite_1_ref': 'Polio reforço',
    'triplice_viral_d1': 'Tríplice Viral D1',
    'triplice_viral_d2': 'Tríplice Viral D2',
    'tetra_viral_src_vz': 'Tetra Viral',
    'dtp': 'DTP',
    'dtp_ref_4_6_anos': 'dTpa reforço 4-6a',
    'triplice_bacte_dtp_1_ref': 'DTP reforço'
}
df_heat = df_heat.rename(columns=nomes_vacinas)

# 8. Heatmap interativo TOP com linha do Brasil no final
fig = px.imshow(
    df_heat.values,
    x=df_heat.columns,
    y=df_heat.index,
    color_continuous_scale="RdYlGn_r",
    range_color=[40, 100],
    text_auto=".1f",
    aspect="auto",
    title="<b>Cobertura Vacinal por Estado – Brasil</b><br><sup>Dados fornecidos por: NTT-DATA | Meta MS ≥ 95% | ≥ 90% aceitável</sup>",
    labels=dict(color="Cobertura (%)"),
    height=960
)

# Deixa os nomes das vacinas na horizontal bonitinho
fig.update_xaxes(tickangle=45)

# Adiciona linha do Brasil no final do gráfico, se existir
if not df_brasil.empty:
    brasil_values = (
        df_brasil[vacina_cols]
        .iloc[0]
        .round(1)
        .rename(nomes_vacinas)
    )
    fig.add_trace(
        px.imshow(
            [brasil_values.values],
            x=brasil_values.index,
            y=["BRASIL"]
        ).data[0]
    )
    fig.data[-1].coloraxis = fig.data[0].coloraxis
    fig.data[-1].showscale = False

# Estilo final
fig.update_layout(
    title_x=0.5,
    font=dict(size=12),
    margin=dict(l=120, t=100),
    plot_bgcolor='white'
)

# Mostra o gráfico interativo lindo
fig.show()
